In [ ]:
import sys
import os
import numpy as np

src_path = os.path.abspath(os.path.join('../../../', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)
    

from tools.data_processor import DataProcessor
from tools.frame_processors import SupervisionVertexProcessorWithLandmarkFrontalization
from tools.frame_preprocessors import TextureFrontalizationPreprocessor, FaceExtractionPreprocessor

In [ ]:
VIDEO= "smile.mp4"
VIDEO_PATH = os.path.abspath(os.path.join('..', 'data', 'expressions', VIDEO))
REFERENCE_POINTS = "key_points_xyz.npy"
REFERENCE_POINTS_PATH = os.path.abspath(os.path.join('..', 'data', 'reference_points', REFERENCE_POINTS))
MODELS_PATH = os.path.join(src_path, "models", "frontalization_models")

In [3]:
class SVPWLF_dummy(SupervisionVertexProcessorWithLandmarkFrontalization):
    KEYPOINTS_HISTORY = []

    def process(self, frame: np.ndarray) -> np.ndarray:
        processed_frame = self._model.process(frame)
        image_to_frontalize_xyz = self._get_xyz_from_processed_frame(processed_frame)
        frontalized_keypoints = self._procrustes_analysis(
            image_to_frontalize_xyz[0], self._reference_points[0]
        )
        frontalized_keypoints = self._get_xy_from_xyz(frontalized_keypoints)
        self.KEYPOINTS_HISTORY.append(frontalized_keypoints)

        return self._make_face_mesh(frontalized_keypoints)

In [ ]:
f_processor = SVPWLF_dummy(
    reference_points_path=REFERENCE_POINTS_PATH
)

dp = DataProcessor(
    frame_processor=f_processor,
    frame_preprocessors=[
        FaceExtractionPreprocessor(
            skip_bad_frames=True,
            output_size=(200, 200)
        ),
        TextureFrontalizationPreprocessor(
            models_path=MODELS_PATH,
            do_calculate_symmetry=True
        ),
    ],
)
dp.process_video(VIDEO_PATH)

 39%|███▊      | 124/321 [00:05<00:08, 22.04it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

: 

In [15]:
len(f_processor.KEYPOINTS_HISTORY[0])

478